In [15]:
# 첫 번째 셀: 라이브러리 임포트 및 환경 설정
from typing import Annotated, List, TypedDict, Dict
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import Pinecone as PineconeVectorStore
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_core.runnables import RunnableConfig
import numpy as np
import pandas as pd
import asyncio
import json
import re

load_dotenv()

True

## * 임베딩 모델 후보(현재는 적재 OpenAI Embedding으로해서, 쿼리도 반드시 같은 임베딩으로 해야함)
- OpenAI Embedding(text-embedding-ada-002) 
- PubMedBERT Embeddings -> PubMed 원문으로 사전학습·문장단위 768-d 임베딩
- BioSimCSE-BioLinkBERT -> BioLinkBERT에 SimCSE 대조학습 적용
- BioBERT-NLI -> BioBERT 기반 + NLI 세트로 파인튜닝
- Multilingual E5-base -> E5 구조·12 L·768 d·다국어 지원
- BMRetriever -> LLM 기반 Bio-전용 dense retriever

## * 리랭커 모델 후보
- MedCPT Cross-Encoder -> PubMedBERT 초기화·18 M 쿼리-문서로 학습
- PubMedBERT Cross-Enc -> PubMed 검색 로그 하드네거티브로 파인튜닝
- BGE reranker-v2-gemma -> Gemma-2 B 백본·다국어 최적화
- monoT5-large -> Seq2Seq pointwise 랭커
- mxbai-rerank-base-v1 -> DeBERTa 125 M 경량 모델
- Rank/ListT5 (11 B) -> Listwise·Fusion-in-Decoder

## * Pinecone 안에서 리트리버 변형하기

In [16]:
# 두 번째 셀: 핵심 Retriever 생성
def create_retriever():
    print("Retriever 생성 중")
    
    # 1536차원 임베딩 사용 (인덱스와 일치)
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    
    # 올바른 설정
    pinecone_vs = PineconeVectorStore.from_existing_index(
        index_name="boazpubmed",
        embedding=embeddings,
        namespace="",  # 빈 네임스페이스 (26483개 문서가 있는 곳)
        text_key="page_content"  # 실제 텍스트가 저장된 키
    )
    
    print("Pinecone 연결 완료")

    # Reranker 설정(Pubmed 기반으로 수정)
    reranker = HuggingFaceCrossEncoder(model_name="ncbi/MedCPT-Cross-Encoder") #MedCPT-Cross-Encoder 리랭커 사용용
    compressor = CrossEncoderReranker(model=reranker, top_n=4)

    base = pinecone_vs.as_retriever(search_kwargs={"k": 10})
    compression_retriever = ContextualCompressionRetriever(
        base_retriever=base,
        base_compressor=compressor
    )
    
    print("Retriever 생성 완료")
    return compression_retriever

# 전역 retriever 생성
retriever = create_retriever()

Retriever 생성 중
Pinecone 연결 완료
Retriever 생성 완료


In [17]:
# 세 번째 셀 ▸ NICHD xls → 사전 & 쿼리 재작성기
import pandas as pd, re, json, sys, subprocess
from pathlib import Path

# 1) 동의어 사전 구축(NIH 제공해주는 Pediatric Terminology.xls 파일 사용) + (UMLS API로 동의어 사전 확장 가능, 아직 미적용)
XLS_PATH = (r"C:\Users\user\Desktop\BOAZ\23기 분석 ADV\소아마취 챗봇 프로젝트"
            r"\소아마취 Agent\Pediatric_Terminology.xls")

def build_term_dict_from_xls(xls_path: str):
    term_dict: Dict[str, List[str]] = {}
    sheets = pd.read_excel(xls_path, sheet_name=None, engine="xlrd")  # 엔진 고정
    for df in sheets.values():
        df = df.rename(columns=lambda c: c.strip())
        if {"Peds Preferred Term", "Peds Synonym"} <= set(df.columns):
            for pref, syns in zip(df["Peds Preferred Term"], df["Peds Synonym"]):
                if pd.isna(pref):
                    continue
                pref = str(pref).strip().lower()
                syn_list: List[str] = []
                if isinstance(syns, str):
                    syn_list = re.split(r"[|;,]", syns)
                elif not pd.isna(syns):
                    syn_list = [str(syns)]
                syn_list = [s.strip().lower() for s in syn_list if s.strip()]
                term_dict.setdefault(pref, []).extend(syn_list + [pref])
    return {k: sorted(set(v)) for k, v in term_dict.items()}

PEDIATRIC_TERMS = build_term_dict_from_xls(XLS_PATH)

# 2) 쿼리 한글 → 영어 선번역 유틸
async def ko2en(text: str, model_name: str = "gpt-4o-mini"):
    """
    한글 의료 질문을 영어로 자연스러운 전문 용어 중심 표현으로 번역
    """
    llm = ChatOpenAI(model_name=model_name, temperature=0.0, max_tokens=300)
    resp = await llm.ainvoke(f"Translate the following pediatric-anesthesia question to English "
                             f"(keep medical terms and abbreviations):\n{text}")
    return resp.content.strip()

# 3) 메인 재작성 클래스
class GPTQueryRewriter:
    """한국어,영어 입력 모두 처리 → 영어 OR 확장 벡터 검색 쿼리 반환"""
    def __init__(self, model_name: str = "gpt-4o-mini", temperature: float = 0.2):
        self.llm = ChatOpenAI(model_name=model_name,
                              temperature=temperature,
                              max_tokens=300)

    async def rewrite(self, question, mode = "initial"):
        # 입력이 한글이면 먼저 영어로 번역
        if not question.isascii():                
            question_en = await ko2en(question)
        else:
            question_en = question

        # 동의어 사전 기반 OR-확장 쿼리 생성
        prompt = (
            "You are a pediatric-anesthesia search expert.\n"
            "Rewrite the following question into ONE concise English search query "
            "optimised for a vector database.\n"
            f"Synonym dictionary: {json.dumps(PEDIATRIC_TERMS, ensure_ascii=False)}\n"
            "Guidelines:\n"
            "① Preserve key medical terms/abbreviations.\n"
            "② Expand synonyms with OR (e.g., neonatal OR newborn).\n"
            "③ Remove stop-words and unnecessary fillers.\n"
            "Return ONLY the search query."
        )
        if mode == "improvement":
            prompt += "\n(Previous results were unsatisfactory. Try a different angle.)"
        elif mode not in {"initial", "improvement"}:
            prompt += f"\n(User feedback: {mode} – please reflect it.)"

        prompt += f"\n\nUser question: {question_en}\n→"

        rewritten = (await self.llm.ainvoke(prompt)).content.strip()
        return rewritten

In [18]:
#  다섯 번째 셀 — PubMed 전용 LLM 평가기 (핵심 3 지표)
"""
- Relevance(관련성) : 쿼리와 증거 간 주제 중복도
- Faithfulness(사실 일치도) : 증거가 질문에 대한 답변을 지지하는지 여부
- Completeness(질문 요소 충족도) : 증거가 질문의 모든 요소를 포함하는지 여부
- G-Eval 원논문과 Confident-AI 해설에서 “Relevance가 인간 평가와 가장 높은 상관(ρ≈0.84)을 보였고 Faithfulness가 그다음(ρ≈0.81)”로 보고되어 두 지표를 0.4·0.35로 가장 크게 반영
- RAGAS·Weaviate 가이드에 따르면 Completeness(답변이 쿼리의 모든 핵심 요소를 다루는지)는 품질에 기여하지만 상관계수는 앞 두 지표보다 낮아 보조 가중치(0.25)로 충분
- 세 가중치 합을 1.0으로 두면 스코어 직관성이 유지되고, 0.6 이상(60 %)을 실무 통과선으로 삼는 최근 RAG 평가 사례들과 동일한 cut-off를 그대로 사용할 수 있다
"""

class LLMEvaluator:
    """Relevance(관련성), Faithfulness(사실 일치도), Completeness(질문 요소 충족도), 세 지표만 계산하는 LLM Judge"""
    def __init__(self, model_name="gpt-4o-mini", temperature=0.0):
        self.judge = ChatOpenAI(model_name=model_name,
                                temperature=temperature,
                                max_tokens=500)

    async def evaluate_search_results(self, query: str, docs: List[str]) -> Dict:
        if not docs:
            return self._default("no documents")

        previews = [d[:200].replace("\n", " ") + ("…" if len(d) > 200 else "")
                    for d in docs[:3]]

        prompt = f"""
            You are an independent evaluator for a PubMed-based RAG system.

            Task ◂ Evaluate how well the retrieved abstracts answer the user query.
            Metrics (0-1):
            1. relevance    – topical overlap between query and abstracts.
            2. faithfulness – factual alignment: does the evidence really support the answer?
            3. completeness – does the evidence cover all key aspects of the question?

            Return **one JSON**:

            {{
            "relevance": 0.0,
            "faithfulness": 0.0,
            "completeness": 0.0,
            "overall": 0.0,
            "feedback": ""
            }}

            overall = 0.4*relevance + 0.35*faithfulness + 0.25*completeness
            User query: \"{query}\"
            Evidence previews:
            - {previews[0]}
            - {previews[1] if len(previews)>1 else ''}
            - {previews[2] if len(previews)>2 else ''}
            """
        
        try:
            raw = (await self.judge.ainvoke(prompt)).content.strip()
            js = re.search(r"\{.*\}", raw, re.S).group()
            data = json.loads(js)
            # 필드 누락 시 기본값 보정
            for k in ("relevance", "faithfulness", "completeness"):
                data.setdefault(k, 0.3)
            # overall 미존재 시 가중합 계산
            if "overall" not in data:
                data["overall"] = round(
                    0.4*data["relevance"] +
                    0.35*data["faithfulness"] +
                    0.25*data["completeness"], 3)
            # 통과 기준(0.6)을 기본값으로 삽입
            data.setdefault("recommended_threshold", 0.6)
            return data
        except Exception as e:
            return self._default(f"parse error: {e}")

    # 모델이 어떤 지표를 빠뜨려도 0.3(보통 수준) 으로 채워 넣어 오류 방지
    def _default(self, reason: str):
        return dict(relevance=0.3, faithfulness=0.3, completeness=0.3,
                    overall=0.3, feedback=reason, recommended_threshold=0.6)

    # 종합 점수에 따른 재검색 여부 결정
    async def should_retry_search(self, ev: Dict, turn: int, max_turn: int):
        return ev.get("overall", 0) < ev.get("recommended_threshold", 0.6) \
               and turn < max_turn


In [19]:
# 여섯 번째 셀: 헬퍼 함수들 (정렬 기준 제거)

gpt_rewriter = GPTQueryRewriter()
llm_evaluator = LLMEvaluator()

# 한글 쿼리를 영어로 선번역 후 동의어 사전을 활용하여 혼합 방식 확장
async def expand_medical_terms(query: str) -> str:
    # 1. 한글이 포함된 경우 영어로 먼저 번역
    if not query.isascii():
        translated_query = await ko2en(query)
        print(f"영어 번역: {translated_query}")
    else:
        translated_query = query
    
    # 2. 번역된 영어 쿼리에서 동의어 사전 매칭하여 혼합 방식 확장
    expanded = translated_query
    translated_lower = translated_query.lower()
    
    for preferred_term, synonyms in PEDIATRIC_TERMS.items():
        # 선호 용어나 동의어가 번역된 쿼리에 포함되어 있는지 확인
        if preferred_term in translated_lower or any(syn in translated_lower for syn in synonyms):
            # 혼합 방식: 가장 관련성 높은 동의어 2-3개만 선택
            relevant_synonyms = select_relevant_synonyms(synonyms, translated_query)
            if relevant_synonyms:
                synonym_phrase = " OR ".join(relevant_synonyms)
                expanded += f" ({synonym_phrase})"
            break  # 첫 번째 매칭만 사용하여 중복 방지
    
    return expanded


def select_relevant_synonyms(synonyms: List[str], original_query: str) -> List[str]:
    """
    동의어 리스트에서 가장 관련성 높은 2-3개 선택 (길이 기준 제거)
    """
    filtered_synonyms = []
    original_lower = original_query.lower()
    
    for syn in synonyms:
        syn_lower = syn.lower()
        # 제외 조건들
        if (len(syn) < 3 or  # 너무 짧은 용어
            len(syn) > 20 or  # 너무 긴 용어
            syn_lower in original_lower or  # 이미 원래 쿼리에 포함된 용어
            syn_lower in ['child', 'children', 'pediatric', 'paediatric']):  # 너무 일반적인 용어
            continue
        filtered_synonyms.append(syn)
    
    # 최대 3개까지만 선택 (순서 유지)
    return filtered_synonyms[:3]

async def multi_strategy_query_expansion(original: str) -> List[str]:
    """
    하이브리드 전략으로 쿼리를 확장 (GPT 재작성 + 동의어 사전 활용)
    """
    strategies = []
    
    # 1. 원본 쿼리 (한글이면 영어로 번역)
    if not original.isascii():
        translated_original = await ko2en(original)
        strategies.append(translated_original)
        print(f"원본 번역: {translated_original}")
    else:
        strategies.append(original)
    
    try:
        # 2. GPT 재작성 (의미적 이해 기반 쿼리 최적화)
        gpt_rewritten = await gpt_rewriter.rewrite(original, "initial")
        strategies.append(gpt_rewritten)
        print(f"GPT 재작성: {gpt_rewritten}")
    except Exception as e:
        print(f"GPT 재작성 실패: {e}")
    
    # 3. 동의어 사전 기반 확장
    medical_expanded = await expand_medical_terms(original)
    strategies.append(medical_expanded)
    print(f"의료 용어 확장: {medical_expanded}")
    
    # 4. 고급 중복 제거 (의미적 유사성 고려)
    unique_strategies = []
    for strategy in strategies:
        if strategy and strategy.strip():
            is_duplicate = False
            for existing in unique_strategies:
                if (strategy.lower() == existing.lower() or 
                    strategy.lower() in existing.lower() or 
                    existing.lower() in strategy.lower()):
                    if len(strategy) > len(existing):
                        unique_strategies.remove(existing)
                        unique_strategies.append(strategy)
                    is_duplicate = True
                    break
            
            if not is_duplicate:
                unique_strategies.append(strategy)
    
    print(f"최종 전략 수: {len(unique_strategies)}")
    return unique_strategies

# 중복 문서 제거 및 랭킹
def remove_duplicates_and_rank(docs: List) -> List:
    seen = set()
    unique_docs = []
    
    for doc in docs:
        content_hash = hash(doc.page_content[:200])
        if content_hash not in seen:
            seen.add(content_hash)
            unique_docs.append(doc)
    
    return unique_docs

# LLMEvaluator를 사용하여 실제 검색 품질 기반으로 최고의 쿼리를 선택
async def select_best_query_with_llm_evaluator(query_variants: List[str], original_question: str) -> tuple[str, Dict]:
    if not query_variants:
        return original_question, {}
    
    if len(query_variants) == 1:
        docs = retriever.invoke(query_variants[0])
        docs_text = [doc.page_content for doc in docs]
        evaluation = await llm_evaluator.evaluate_search_results(query_variants[0], docs_text)
        return query_variants[0], evaluation
    
    print(f"{len(query_variants)}개 쿼리 변형에 대해 검색 품질 평가 시작")
    
    best_query = query_variants[0]
    best_score = 0
    best_evaluation = {}
    evaluations = []
    
    for i, query in enumerate(query_variants):
        try:
            print(f"쿼리 {i+1}/{len(query_variants)}: {query}")
            docs = retriever.invoke(query)
            docs_text = [doc.page_content for doc in docs]
            evaluation = await llm_evaluator.evaluate_search_results(query, docs_text)
            evaluations.append((query, evaluation))
            overall_score = evaluation.get("overall", 0)
            print(f"평가 점수: {overall_score:.3f}")
            print(f"세부점수 - 관련성:{evaluation.get('relevance', 0):.2f} "
                  f"일치도:{evaluation.get('faithfulness', 0):.2f} "
                  f"완성도:{evaluation.get('completeness', 0):.2f}")
            
            if overall_score > best_score:
                best_score = overall_score
                best_query = query
                best_evaluation = evaluation
                print(f"현재 최고 쿼리 업데이트!")
        except Exception as e:
            print(f"쿼리 평가 중 오류: {e}")
            default_eval = {
                "relevance": 0.3,
                "faithfulness": 0.3, 
                "completeness": 0.3,
                "overall": 0.3,
                "feedback": f"평가 오류: {str(e)[:50]}",
                "recommended_threshold": 0.6
            }
            evaluations.append((query, default_eval))
    
    print(f"최종 선택된 쿼리: {best_query}")
    print(f"최고 점수: {best_score:.3f}")
    print(f"선택 근거: {best_evaluation.get('feedback', '평가 완료')}")
    
    print("\n전체 쿼리 평가 결과:")
    for query, eval_result in evaluations:
        score = eval_result.get("overall", 0)
        print(f"  {score:.3f} | {query}")
    
    return best_query, best_evaluation

async def select_multiple_best_queries_with_evaluation(query_variants: List[str], original_question: str, top_k: int = 2) -> List[tuple[str, Dict]]:
    """
    LLMEvaluator 기반으로 상위 K개의 쿼리와 평가 결과를 반환
    """
    if not query_variants or len(query_variants) <= top_k:
        results = []
        for query in query_variants or [original_question]:
            try:
                docs = retriever.invoke(query)
                docs_text = [doc.page_content for doc in docs]
                evaluation = await llm_evaluator.evaluate_search_results(query, docs_text)
                results.append((query, evaluation))
            except Exception as e:
                default_eval = {"overall": 0.3, "feedback": f"오류: {e}"}
                results.append((query, default_evaluator))
        return results
    
    print(f"상위 {top_k}개 쿼리 선택을 위한 전체 평가 시작")
    
    scored_queries = []
    for query in query_variants:
        try:
            docs = retriever.invoke(query)
            docs_text = [doc.page_content for doc in docs]
            evaluation = await llm_evaluator.evaluate_search_results(query, docs_text)
            overall_score = evaluation.get("overall", 0)
            scored_queries.append((query, evaluation, overall_score))
        except Exception as e:
            default_eval = {
                "overall": 0.3, 
                "feedback": f"평가 오류: {str(e)[:50]}",
                "relevance": 0.3, "faithfulness": 0.3, "completeness": 0.3
            }
            scored_queries.append((query, default_eval, 0.3))
    
    scored_queries.sort(key=lambda x: x[2], reverse=True)
    selected = [(query, evaluation) for query, evaluation, score in scored_queries[:top_k]]
    
    print(f"상위 {top_k}개 쿼리 선택 완료:")
    for i, (query, evaluation) in enumerate(selected):
        print(f"  {i+1}. {evaluation.get('overall', 0):.3f} | {query}")
    
    return selected

print("헬퍼 함수들 준비 완료")

헬퍼 함수들 준비 완료


In [20]:
# 일곱 번째 셀: State 및 모델 설정
MAX_RETRY = 4

class ChatbotState(TypedDict):
    original_question: Annotated[str, "Original_Question"]
    current_query: Annotated[str, "Current_Query"]
    query_variants: Annotated[List[str], "Query_Variants"]
    vector_documents: Annotated[str, "Vector_Documents"]
    graph_db_context: Annotated[str, "Graph_DB_Context"]
    llm_evaluation: Annotated[Dict, "LLM_Evaluation"]
    loop_cnt: Annotated[int, "Loop_Count"]
    final_answer: Annotated[str, "Final_Answer"]
    messages: Annotated[List, add_messages]

memory = MemorySaver()

LLM_SYSTEM_PROMPT = """# 소아마취학 전문가 AI 시스템

## 역할 정의
당신은 소아마취학 전문가 AI로서, 임상 의사결정 지원 시스템(Clinical Decision Support System)의 역할을 합니다. 
신생아부터 청소년까지의 소아 환자 마취 관리에 대한 전문적이고 근거 기반의 임상 정보를 제공합니다.

## 의사소통 원칙
- 명확하고 실용적인 정보 제공
- 근거 기반 권고사항 명시
- 불확실한 경우 추가 전문의 상담 권고
- 안전성을 최우선으로 고려

## 컨텍스트 정보

### Vector DB 검색 결과
{VectorDB}

### 그래프 DB 환자 정보 (해당하는 경우)
{GraphDB}

## 사용자 질문
{question}

"""

chat_model = ChatOpenAI(temperature=0.2, model_name="gpt-4o-mini")

# 전역 변수들이 제대로 생성되었는지 확인
try:
    print(f"gpt_rewriter 상태: {type(gpt_rewriter).__name__}")
    print(f"llm_evaluator 상태: {type(llm_evaluator).__name__}")
    print(f"retriever 상태: {type(retriever).__name__}")
    print(f"chat_model 상태: {type(chat_model).__name__}")
except NameError as e:
    print(f"변수 정의 오류: {e}")

print("State 및 모델 설정 완료")

gpt_rewriter 상태: GPTQueryRewriter
llm_evaluator 상태: LLMEvaluator
retriever 상태: ContextualCompressionRetriever
chat_model 상태: ChatOpenAI
State 및 모델 설정 완료


In [21]:
# 개선된 gpt_query_rewriter_node (LLMEvaluator 기반 Best Query 선택)
async def gpt_query_rewriter_node(state: ChatbotState):
    """GPT를 이용한 쿼리 재작성 노드 (LLMEvaluator 기반 Best Query 선택)"""
    loop_cnt = state.get("loop_cnt", 0)
    print(f"쿼리 재작성 노드 - 시도 {loop_cnt + 1}")
    
    try:
        if loop_cnt == 0:
            # 첫 번째 시도: 다중 전략 쿼리 확장
            query_variants = await multi_strategy_query_expansion(state["original_question"])
            print(f"생성된 쿼리 변형들: {query_variants}")
            
            # LLMEvaluator 기반 best query 선택
            best_query, best_evaluation = await select_best_query_with_llm_evaluator(
                query_variants, 
                state["original_question"]
            )
            print(f"LLM 평가 기반으로 선택된 최고 쿼리: {best_query}")
            print(f"선택된 쿼리의 점수: {best_evaluation.get('overall', 0):.3f}")
            
        elif loop_cnt == 1:
            # 두 번째 시도: 기존 변형 중 다른 쿼리 시도
            existing_variants = state.get("query_variants", [state["original_question"]])
            if len(existing_variants) > 1:
                # 상위 2개 쿼리 중 두 번째 선택
                selected_queries = await select_multiple_best_queries_with_evaluation(
                    existing_variants, state["original_question"], top_k=2
                )
                if len(selected_queries) > 1:
                    best_query, best_evaluation = selected_queries[1]  # 두 번째 최고 쿼리
                    print(f"두 번째 최고 쿼리 선택: {best_query}")
                else:
                    best_query, best_evaluation = selected_queries[0]
                    print(f"첫 번째 쿼리 재사용: {best_query}")
            else:
                best_query = existing_variants[0] if existing_variants else state["original_question"]
                print(f"기존 쿼리 사용: {best_query}")
                
        elif loop_cnt == 2:
            # 세 번째 시도: 다양성 기반 선택 또는 새로운 재작성
            existing_variants = state.get("query_variants", [state["original_question"]])
            try:
                best_query = await gpt_rewriter.rewrite(state['original_question'], "improvement")
                print(f"개선 기반 재작성: {best_query}")
            except:
                best_query = existing_variants[-1] if existing_variants else state["original_question"]
                print(f"마지막 변형 사용: {best_query}")
                
        else:
            # 마지막 시도: 원본 질문 사용
            best_query = state["original_question"]
            print(f"원본 질문 사용: {best_query}")
        
        return ChatbotState(
            current_query=best_query,
            query_variants=await multi_strategy_query_expansion(state["original_question"]) if loop_cnt == 0 else state.get("query_variants", []),
            loop_cnt=loop_cnt + 1
        )
    except Exception as e:
        print(f"쿼리 재작성 중 오류: {e}")
        # 오류 시 원본 질문 사용
        return ChatbotState(
            current_query=state["original_question"],
            query_variants=[state["original_question"]],
            loop_cnt=loop_cnt + 1
        )

print("개선된 LLMEvaluator 기반 Best Query 선택 로직 적용 완료!")


개선된 LLMEvaluator 기반 Best Query 선택 로직 적용 완료!


In [22]:
# 여덟 번째 셀: LangGraph 노드들
async def gpt_query_rewriter_node(state: ChatbotState):
    """GPT를 이용한 쿼리 재작성 노드 (LLMEvaluator 기반 Best Query 선택)"""
    loop_cnt = state.get("loop_cnt", 0)
    try:
        if loop_cnt == 0:
            query_variants = await multi_strategy_query_expansion(state["original_question"])
            best_query, best_evaluation = await select_best_query_with_llm_evaluator(
                query_variants, state["original_question"]
            )
        elif loop_cnt == 1:
            existing_variants = state.get("query_variants", [state["original_question"]])
            selected = await select_multiple_best_queries_with_evaluation(
                existing_variants, state["original_question"], top_k=2
            )
            best_query = selected[1][0] if len(selected) > 1 else selected[0][0]
        elif loop_cnt == 2:
            try:
                best_query = await gpt_rewriter.rewrite(state['original_question'], "improvement")
            except:
                best_query = state["query_variants"][-1]
        else:
            best_query = state["original_question"]
        return ChatbotState(
            current_query=best_query,
            query_variants=(query_variants if loop_cnt == 0 else state.get("query_variants", [])),
            loop_cnt=loop_cnt + 1
        )
    except Exception as e:
        return ChatbotState(
            current_query=state["original_question"],
            query_variants=[state["original_question"]],
            loop_cnt=loop_cnt+1
        )

async def ensemble_search_node(state: ChatbotState):
    """Vector DB에서 문서 검색을 수행하는 노드"""
    current_query = state.get("current_query")
    docs = retriever.invoke(current_query)
    result_text = "\n".join([d.page_content for d in docs])
    return ChatbotState(vector_documents=result_text)

async def llm_evaluation_node(state: ChatbotState):
    """검색 결과의 품질을 평가하는 노드"""
    docs_list = [d for d in state.get("vector_documents", "").split("\n") if d]
    if not docs_list:
        return ChatbotState(llm_evaluation=LLMEvaluator()._default("no docs"))
    evaluation = await llm_evaluator.evaluate_search_results(state.get("current_query"), docs_list)
    for field in ["relevance","faithfulness","completeness","overall","feedback","recommended_threshold"]:
        evaluation.setdefault(field, 0.3 if field!="feedback" else "missing")
    return ChatbotState(llm_evaluation=evaluation)

async def merge_outputs(state: ChatbotState):
    """검색 결과와 평가 피드백을 바탕으로 최종 답변을 생성하는 노드"""
    eval = state.get("llm_evaluation", {})
    overall = eval.get("overall", 0)
    threshold = eval.get("recommended_threshold", 0.6)
    # 평가 점수 미달 시 일반화된 안내 메시지
    if overall < threshold:
        fallback = (
            "현재 문헌 검색으로는 질문하신 사항에 정확히 답변드리기 어렵습니다. "
            "유사한 일반 소아 KMS 치료 지침을 참고하시거나, 질문을 구체화하여 다시 문의해 주세요."
        )
        return ChatbotState(final_answer=fallback, messages=[("assistant", fallback)])
    # 충분한 자료 확보 시 LLM을 이용해 상세 답변 생성
    context = state.get("vector_documents", "")
    prompt = LLM_SYSTEM_PROMPT.format(
        VectorDB=context,
        GraphDB=state.get("graph_db_context", ""),
        question=state.get("original_question")
    )
    response = chat_model.invoke(prompt)
    answer = response.content or "죄송합니다. 답변을 생성할 수 없습니다."
    return ChatbotState(final_answer=answer, messages=[("assistant", answer)])

print("LangGraph 노드들 준비 완료")

LangGraph 노드들 준비 완료


In [23]:
# 아홉 번째 셀: LangGraph 빌드
sg = StateGraph(ChatbotState)
sg.add_node("gpt_query_rewriter", gpt_query_rewriter_node)
sg.add_node("ensemble_search", ensemble_search_node)
sg.add_node("llm_evaluation", llm_evaluation_node)
sg.add_node("merge_outputs", merge_outputs)

sg.add_edge(START, "gpt_query_rewriter")
sg.add_edge("gpt_query_rewriter", "ensemble_search")
sg.add_edge("ensemble_search", "llm_evaluation")

async def _llm_based_route(state: ChatbotState):
    """평가 결과에 따른 라우팅 결정 함수"""
    evaluation = state.get("llm_evaluation", {})
    loop_cnt = state.get("loop_cnt", 0)
    
    # 평가 정보 로깅
    overall_score = evaluation.get("overall", 0)
    threshold = evaluation.get("recommended_threshold", 0.6)
    print(f"라우팅 평가: 점수 {overall_score:.3f} vs 임계값 {threshold:.3f}")
    print(f"현재 시도 횟수: {loop_cnt}/{MAX_RETRY}")
    
    try:
        should_retry = await llm_evaluator.should_retry_search(evaluation, loop_cnt, MAX_RETRY)
        result = "retry_rewrite" if should_retry else "generate_answer"
        
        if should_retry:
            print(f"품질 부족으로 재시도: {result}")
            print(f"피드백: {evaluation.get('feedback', '피드백 없음')}")
        else:
            print(f"품질 충족 또는 최대 시도 도달: {result}")
        
        return result
        
    except Exception as e:
        print(f"라우팅 결정 중 오류: {e}")
        # 오류 시 답변 생성으로 진행
        return "generate_answer"

sg.add_conditional_edges(
    "llm_evaluation",
    _llm_based_route,
    {
        "retry_rewrite": "gpt_query_rewriter",
        "generate_answer": "merge_outputs",
    },
)

sg.add_edge("merge_outputs", END)

graph = sg.compile(checkpointer=memory)

print("LangGraph 빌드 완료")

LangGraph 빌드 완료


In [24]:
# 열 번째 셀: 실행 함수
async def run_chatbot(question: str, show_details: bool = True, graph_db_context: str = ""):
    """Jupyter용 챗봇 실행 함수"""
    config = RunnableConfig(configurable={"thread_id": 1})
    
    init_state = {
        "original_question": question,
        "current_query": "",
        "query_variants": [],
        "vector_documents": "",
        "graph_db_context": graph_db_context,
        "llm_evaluation": {},
        "loop_cnt": 0,
        "final_answer": "",
        "messages": [],
    }
    
    print(f"원본 질문: {question}")
    print("=" * 100)
    
    final_result = None
    
    async for event in graph.astream(init_state, config=config):
        for node_name, node_state in event.items():
            print(f"\n현재 노드: {node_name}")
            
            if show_details:
                if node_name == "gpt_query_rewriter":
                    print(f"GPT 재작성된 쿼리 (시도 {node_state['loop_cnt']}): {node_state['current_query']}")
                    
                elif node_name == "llm_evaluation":
                    evaluation = node_state['llm_evaluation']
                    print(f"LLM 평가 결과 (시도 {node_state.get('loop_cnt', 0)}):")
                    print(f"   관련성: {evaluation.get('relevance', 0):.3f}")
                    print(f"   사실 일치도: {evaluation.get('faithfulness', 0):.3f}")
                    print(f"   정보 완성도: {evaluation.get('completeness', 0):.3f}")
                    print(f"   종합 점수: {evaluation.get('overall', 0):.3f}")
                    print(f"   권장 임계치: {evaluation.get('recommended_threshold', 0):.3f}")
                    
                    if evaluation.get('overall', 0) < evaluation.get('recommended_threshold', 0.6):
                        print("   점수가 낮아 쿼리를 다시 재작성합니다.")
                        print(f"   피드백: {evaluation.get('feedback', '')}")
                    else:
                        print("   점수가 충분합니다.")
                    print("-" * 80)
                    
                elif node_name == "merge_outputs":
                    print("최종 답변:")
                    print(node_state['final_answer'])
                    final_result = node_state
    
    return final_result

print("실행 함수 준비 완료")

실행 함수 준비 완료


In [25]:
# 그래프 재빌드 (업데이트된 함수들 적용)
try:
    del graph  # 기존 그래프 삭제
except:
    pass

sg = StateGraph(ChatbotState)
sg.add_node("gpt_query_rewriter", gpt_query_rewriter_node)
sg.add_node("ensemble_search", ensemble_search_node)
sg.add_node("llm_evaluation", llm_evaluation_node)
sg.add_node("merge_outputs", merge_outputs)

sg.add_edge(START, "gpt_query_rewriter")
sg.add_edge("gpt_query_rewriter", "ensemble_search")
sg.add_edge("ensemble_search", "llm_evaluation")

async def _llm_based_route(state: ChatbotState):
    """평가 결과에 따른 라우팅 결정 함수"""
    evaluation = state.get("llm_evaluation", {})
    loop_cnt = state.get("loop_cnt", 0)
    
    # 평가 정보 로깅
    overall_score = evaluation.get("overall", 0)
    threshold = evaluation.get("recommended_threshold", 0.6)
    print(f"라우팅 평가: 점수 {overall_score:.3f} vs 임계값 {threshold:.3f}")
    print(f"현재 시도 횟수: {loop_cnt}/{MAX_RETRY}")
    
    try:
        should_retry = await llm_evaluator.should_retry_search(evaluation, loop_cnt, MAX_RETRY)
        result = "retry_rewrite" if should_retry else "generate_answer"
        
        if should_retry:
            print(f"품질 부족으로 재시도: {result}")
            print(f"피드백: {evaluation.get('feedback', '피드백 없음')}")
        else:
            print(f"품질 충족 또는 최대 시도 도달: {result}")
        
        return result
        
    except Exception as e:
        print(f"라우팅 결정 중 오류: {e}")
        # 오류 시 답변 생성으로 진행
        return "generate_answer"

sg.add_conditional_edges(
    "llm_evaluation",
    _llm_based_route,
    {
        "retry_rewrite": "gpt_query_rewriter",
        "generate_answer": "merge_outputs",
    },
)

sg.add_edge("merge_outputs", END)

graph = sg.compile(checkpointer=memory)

print("LLMEvaluator 기반 그래프 빌드 완료")


LLMEvaluator 기반 그래프 빌드 완료


In [27]:
# 열한 번째 셀: 실행 (Jupyter에서 await 직접 사용)
# question = "Kasabach-Merritt Syndrome의 일반적 치료법은?"
question = "3세 Kasabach-Merritt Syndrome 환자의 일반적 치료법은?"
result = await run_chatbot(question)
print("\n실행 완료!")

원본 질문: 3세 Kasabach-Merritt Syndrome 환자의 일반적 치료법은?
원본 번역: What is the general treatment method for a 3-year-old patient with Kasabach-Merritt Syndrome?
GPT 재작성: treatment OR management AND 3-year-old OR pediatric AND Kasabach-Merritt Syndrome
영어 번역: What is the general treatment for a 3-year-old patient with Kasabach-Merritt Syndrome?
의료 용어 확장: What is the general treatment for a 3-year-old patient with Kasabach-Merritt Syndrome?
최종 전략 수: 3
3개 쿼리 변형에 대해 검색 품질 평가 시작
쿼리 1/3: What is the general treatment method for a 3-year-old patient with Kasabach-Merritt Syndrome?
평가 점수: 0.485
세부점수 - 관련성:0.50 일치도:0.60 완성도:0.40
현재 최고 쿼리 업데이트!
쿼리 2/3: treatment OR management AND 3-year-old OR pediatric AND Kasabach-Merritt Syndrome
평가 점수: 0.485
세부점수 - 관련성:0.50 일치도:0.70 완성도:0.40
쿼리 3/3: What is the general treatment for a 3-year-old patient with Kasabach-Merritt Syndrome?
평가 점수: 0.485
세부점수 - 관련성:0.50 일치도:0.60 완성도:0.40
최종 선택된 쿼리: What is the general treatment method for a 3-year-old patient with Kasabach-M